In [1]:
from datetime import datetime, timedelta
#import sys

#import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import yfinance as yf

#sys.path.append('..')
#from util.batcher import Batcher
#from util.stock_loader import Loader

In [30]:
SYMBOLS = [
    'ABBV', 'APA',
    'AXON', 'CF', 'COST', 'CVX', 'DVN', 'EDIT', 'ETSY', 'EXC', 'FLGT',
    'GH', 'GMED', 'GOOG', 'HSY', 'INTG', 'ISRG', 'JD', 'LLY', 'LSPD',
    'LULU', 'MCK', 'MDB', 'MELI', 'MNST', 'MPC', 'MSFT', 'MTCH', 'NVCR', 'NUE', 'NVDA',
    'OKTA', 'OPEN', 'OXY', 'PANW', 'PINS', 'PLD', 'PSA', 'PTON', 'QDEL', 'RBLX',
    'RDFN', 'RGEN', 'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW',
    'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR',
    'UPST', 'VEEV', 'XPEV', 'ZEN', 'ZNGA']

In [31]:
TODAY = datetime.now().date()
N_YEARS = 10
START = TODAY - timedelta(round(365.25 * N_YEARS))
START, TODAY

(datetime.date(2012, 4, 30), datetime.date(2022, 4, 30))

In [32]:
df = yf.download(SYMBOLS, start=START, end=TODAY)
df.drop(
    columns=['Open', 'Close', 'Volume', 'High', 'Low'], inplace=True)
df.columns = SYMBOLS
print(df.shape)
df.head()

[*********************100%***********************]  61 of 61 completed
(2518, 61)


,ABBV,APA,AXON,CF,COST,CVX,DVN,EDIT,ETSY,EXC,...,TREX,TTD,TTWO,TWLO,TWTR,UPST,VEEV,XPEV,ZEN,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-30,NaN,80.846146,4.60,29.727098,67.283104,70.426613,54.846912,NaN,NaN,19.019388,...,4.00000,NaN,14.11,NaN,NaN,NaN,NaN,NaN,NaN,8.34
2012-05-01,NaN,82.388214,4.64,30.058153,66.359840,71.556755,55.475109,NaN,NaN,19.160782,...,3.97250,NaN,13.86,NaN,NaN,NaN,NaN,NaN,NaN,8.46
2012-05-02,NaN,80.509064,4.90,30.815733,66.108055,70.737236,53.441406,NaN,NaN,19.151028,...,4.12125,NaN,13.58,NaN,NaN,NaN,NaN,NaN,NaN,8.96
2012-05-03,NaN,77.492302,4.92,30.407690,64.276787,70.049896,52.192921,NaN,NaN,18.926756,...,4.03750,NaN,13.65,NaN,NaN,NaN,NaN,NaN,NaN,8.65
2012-05-04,NaN,75.048576,4.76,28.318205,63.628239,68.549629,51.062206,NaN,NaN,18.785362,...,3.99375,NaN,13.41,NaN,NaN,NaN,NaN,NaN,NaN,8.33


In [33]:
n_nan = df.isnull().sum()
df = df[n_nan[n_nan < 1250].index]
df.head()

,ABBV,APA,AXON,CF,COST,CVX,DVN,EDIT,ETSY,EXC,...,TDOC,TEAM,TREX,TTD,TTWO,TWLO,TWTR,VEEV,ZEN,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-30,NaN,80.846146,4.60,29.727098,67.283104,70.426613,54.846912,NaN,NaN,19.019388,...,NaN,NaN,4.00000,NaN,14.11,NaN,NaN,NaN,NaN,8.34
2012-05-01,NaN,82.388214,4.64,30.058153,66.359840,71.556755,55.475109,NaN,NaN,19.160782,...,NaN,NaN,3.97250,NaN,13.86,NaN,NaN,NaN,NaN,8.46
2012-05-02,NaN,80.509064,4.90,30.815733,66.108055,70.737236,53.441406,NaN,NaN,19.151028,...,NaN,NaN,4.12125,NaN,13.58,NaN,NaN,NaN,NaN,8.96
2012-05-03,NaN,77.492302,4.92,30.407690,64.276787,70.049896,52.192921,NaN,NaN,18.926756,...,NaN,NaN,4.03750,NaN,13.65,NaN,NaN,NaN,NaN,8.65
2012-05-04,NaN,75.048576,4.76,28.318205,63.628239,68.549629,51.062206,NaN,NaN,18.785362,...,NaN,NaN,3.99375,NaN,13.41,NaN,NaN,NaN,NaN,8.33


In [34]:
mu = mean_historical_return(df)
s = CovarianceShrinkage(df).ledoit_wolf()

In [35]:
ef = EfficientFrontier(mu, s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
for k, v in dict(cleaned_weights).items():
    print(f'{k:6s}: {v}')

ABBV  : 0.03706
APA   : 0.0
AXON  : 0.01205
CF    : 0.0
COST  : 0.22677
CVX   : 0.0
DVN   : 0.0
EDIT  : 0.0
ETSY  : 0.0
EXC   : 0.0
FLGT  : 0.01592
GMED  : 0.0
GOOG  : 0.0
HSY   : 0.08014
INTG  : 0.03307
ISRG  : 0.0
JD    : 0.0
LLY   : 0.14921
LULU  : 0.0
MCK   : 0.0
MELI  : 0.0
MNST  : 0.0
MPC   : 0.0
MSFT  : 0.0
MTCH  : 0.0
NVCR  : 0.0
NUE   : 0.0
NVDA  : 0.06782
OKTA  : 0.0
OXY   : 0.0
PANW  : 0.0
PLD   : 0.0
PSA   : 0.0
QDEL  : 0.0
RGEN  : 0.05291
SHOP  : 0.07063
TDOC  : 0.0
TEAM  : 0.11494
TREX  : 0.0
TTD   : 0.13948
TTWO  : 0.0
TWLO  : 0.0
TWTR  : 0.0
VEEV  : 0.0
ZEN   : 0.0
ZNGA  : 0.0


In [36]:
_, _, s1 = ef.portfolio_performance(verbose=True)

Expected annual return: 37.3%
Annual volatility: 19.6%
Sharpe Ratio: 1.80


In [8]:
retain_1 = {k: v for k, v in dict(cleaned_weights).items() if v}
retain_1

{'ADYEY': 0.21442,
 'ANET': 0.00412,
 'BROS': 0.40423,
 'DDOG': 0.07456,
 'INTG': 0.02981,
 'MDB': 0.06359,
 'NVDA': 0.01727,
 'RGEN': 0.01857,
 'SHOP': 0.00421,
 'TTD': 0.0248,
 'TTWO': 0.00445,
 'UPST': 0.1399}

In [9]:
df.drop(columns=retain_1.keys(), inplace=True)
df.head()

,AAPL,ABMD,ABNB,AMD,AMZN,AXON,BYND,CHWY,COIN,DOCS,...,TASK,TDOC,TEAM,TREX,TWLO,TWTR,VEEV,XPEV,ZEN,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-30,17.858816,24.330000,NaN,7.36,231.899994,4.60,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.00000,NaN,NaN,NaN,NaN,NaN,8.34
2012-05-01,17.802252,24.500000,NaN,7.59,230.039993,4.64,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.97250,NaN,NaN,NaN,NaN,NaN,8.46
2012-05-02,17.919979,23.900000,NaN,7.63,230.250000,4.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.12125,NaN,NaN,NaN,NaN,NaN,8.96
2012-05-03,17.792767,23.740000,NaN,7.41,229.449997,4.92,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.03750,NaN,NaN,NaN,NaN,NaN,8.65
2012-05-04,17.286034,23.209999,NaN,7.18,223.990005,4.76,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.99375,NaN,NaN,NaN,NaN,NaN,8.33


In [10]:
mu = mean_historical_return(df)
s = CovarianceShrinkage(df).ledoit_wolf()
ef = EfficientFrontier(mu, s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
for k, v in dict(cleaned_weights).items():
    print(f'{k:6s}: {v}')

AAPL  : 0.05229
ABMD  : 0.03873
ABNB  : 0.0
AMD   : 0.0
AMZN  : 0.0493
AXON  : 0.07962
BYND  : 0.0
CHWY  : 0.0
COIN  : 0.0
DOCS  : 0.0
DOCU  : 0.0
DUOL  : 0.0
EDIT  : 0.0
ETSY  : 0.0
FLGT  : 0.04404
GH    : 0.0
GMED  : 0.01739
GOOG  : 0.00486
ISRG  : 0.0
JD    : 0.0
LSPD  : 0.0
LULU  : 0.0
MELI  : 0.0
MNST  : 0.0
MSFT  : 0.13169
MTCH  : 0.0
NVCR  : 0.0
OKTA  : 0.11366
OPEN  : 0.0
PANW  : 0.00591
PINS  : 0.0
PTON  : 0.0
QDEL  : 0.0
RBLX  : 0.0
RDFN  : 0.0
ROKU  : 0.0
RVLV  : 0.0
SE    : 0.16895
SNOW  : 0.0
TASK  : 0.0
TDOC  : 0.0
TEAM  : 0.26476
TREX  : 0.021
TWLO  : 0.0
TWTR  : 0.0
VEEV  : 0.0
XPEV  : 0.0
ZEN   : 0.00779
ZNGA  : 0.0


In [11]:
_, _, s2 = ef.portfolio_performance(verbose=True)

Expected annual return: 36.9%
Annual volatility: 24.2%
Sharpe Ratio: 1.44


In [12]:
retain_2 = {k: v for k, v in dict(cleaned_weights).items() if v}
retain_2

{'AAPL': 0.05229,
 'ABMD': 0.03873,
 'AMZN': 0.0493,
 'AXON': 0.07962,
 'FLGT': 0.04404,
 'GMED': 0.01739,
 'GOOG': 0.00486,
 'MSFT': 0.13169,
 'OKTA': 0.11366,
 'PANW': 0.00591,
 'SE': 0.16895,
 'TEAM': 0.26476,
 'TREX': 0.021,
 'ZEN': 0.00779}

In [13]:
df.drop(columns=retain_2.keys(), inplace=True)
df.head()

,ABNB,AMD,BYND,CHWY,COIN,DOCS,DOCU,DUOL,EDIT,ETSY,...,ROKU,RVLV,SNOW,TASK,TDOC,TWLO,TWTR,VEEV,XPEV,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-30,NaN,7.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.34
2012-05-01,NaN,7.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.46
2012-05-02,NaN,7.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.96
2012-05-03,NaN,7.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.65
2012-05-04,NaN,7.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33


In [14]:
mu = mean_historical_return(df)
s = CovarianceShrinkage(df).ledoit_wolf()
ef = EfficientFrontier(mu, s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
for k, v in dict(cleaned_weights).items():
    print(f'{k:6s}: {v}')

ABNB  : 0.0
AMD   : 0.06969
BYND  : 0.0
CHWY  : 0.0
COIN  : 0.0
DOCS  : 0.0
DOCU  : 0.0496
DUOL  : 0.0
EDIT  : 0.0
ETSY  : 0.0
GH    : 0.09001
ISRG  : 0.0
JD    : 0.0
LSPD  : 0.0
LULU  : 0.03822
MELI  : 0.07653
MNST  : 0.15535
MTCH  : 0.08294
NVCR  : 0.02881
OPEN  : 0.0
PINS  : 0.0
PTON  : 0.0
QDEL  : 0.08168
RBLX  : 0.0
RDFN  : 0.0
ROKU  : 0.19839
RVLV  : 0.0
SNOW  : 0.0
TASK  : 0.0
TDOC  : 0.0
TWLO  : 0.058
TWTR  : 0.0
VEEV  : 0.07078
XPEV  : 0.0
ZNGA  : 0.0


In [15]:
_, _, s3 = ef.portfolio_performance(verbose=True)

Expected annual return: 23.5%
Annual volatility: 25.5%
Sharpe Ratio: 0.84


In [16]:
retain_3 = {k: v for k, v in dict(cleaned_weights).items() if v}
retain_3

{'AMD': 0.06969,
 'DOCU': 0.0496,
 'GH': 0.09001,
 'LULU': 0.03822,
 'MELI': 0.07653,
 'MNST': 0.15535,
 'MTCH': 0.08294,
 'NVCR': 0.02881,
 'QDEL': 0.08168,
 'ROKU': 0.19839,
 'TWLO': 0.058,
 'VEEV': 0.07078}

In [17]:
df.drop(columns=retain_3.keys(), inplace=True)
df.head()

,ABNB,BYND,CHWY,COIN,DOCS,DUOL,EDIT,ETSY,ISRG,JD,...,PTON,RBLX,RDFN,RVLV,SNOW,TASK,TDOC,TWTR,XPEV,ZNGA
Date,,,,,,,,,,,,,,,,,,,,,
2012-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.244446,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.34
2012-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.364441,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.46
2012-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.128891,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.96
2012-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.263336,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.65
2012-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.795555,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33


In [18]:
mu = mean_historical_return(df)
s = CovarianceShrinkage(df).ledoit_wolf()
ef = EfficientFrontier(mu, s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
for k, v in dict(cleaned_weights).items():
    print(f'{k:6s}: {v}')

ABNB  : 0.0
BYND  : 0.0
CHWY  : 0.0
COIN  : 0.0
DOCS  : 0.0
DUOL  : 0.0
EDIT  : 0.0
ETSY  : 0.18457
ISRG  : 0.49619
JD    : 0.20629
LSPD  : 0.0
OPEN  : 0.0
PINS  : 0.0
PTON  : 0.0
RBLX  : 0.0
RDFN  : 0.0
RVLV  : 0.00718
SNOW  : 0.0
TASK  : 0.0
TDOC  : 0.0
TWTR  : 0.0
XPEV  : 0.10577
ZNGA  : 0.0


In [19]:
_, _, s4 = ef.portfolio_performance(verbose=True)

Expected annual return: 14.3%
Annual volatility: 25.8%
Sharpe Ratio: 0.47


In [20]:
retain_4 = {k: v for k, v in dict(cleaned_weights).items() if v}
retain_4

{'ETSY': 0.18457,
 'ISRG': 0.49619,
 'JD': 0.20629,
 'RVLV': 0.00718,
 'XPEV': 0.10577}

In [21]:
s1, s2, s3, s4

(3.0499352541024973, 1.443391850446067, 0.8443250399447338, 0.4743404185668627)

In [22]:
w = np.array([s1, s2, s3, s4])
w = w / w.sum()
w

array([0.52476586, 0.24834716, 0.1452729 , 0.08161408])

In [23]:
retain_1 = {k: v * w[0] for k, v in retain_1.items()}
retain_1

{'ADYEY': 0.11252029490559555,
 'ANET': 0.0021620353279127587,
 'BROS': 0.21212610208790636,
 'DDOG': 0.03912654224494545,
 'INTG': 0.015643270175990126,
 'MDB': 0.03336986080144958,
 'NVDA': 0.009062706338119743,
 'RGEN': 0.009744901951296098,
 'SHOP': 0.002209264254978814,
 'TTD': 0.01301419323597971,
 'TTWO': 0.002335208060488295,
 'UPST': 0.07341474329490168}

In [24]:
retain_2 = {k: v * w[1] for k, v in retain_2.items()}
retain_2

{'AAPL': 0.012986073027609893,
 'ABMD': 0.009618485529916449,
 'AMZN': 0.012243515017425275,
 'AXON': 0.019773400926722116,
 'FLGT': 0.010937208952685785,
 'GMED': 0.004318757122779422,
 'GOOG': 0.0012069672005007473,
 'MSFT': 0.0327048375790007,
 'OKTA': 0.028227138273439284,
 'PANW': 0.001467731719127452,
 'SE': 0.04195825278283976,
 'TEAM': 0.06575239423962506,
 'TREX': 0.005215290372534093,
 'ZEN': 0.0019346243810495517}

In [25]:
retain_3 = {k: v * w[2] for k, v in retain_3.items()}
retain_3

{'AMD': 0.0101240687071298,
 'DOCU': 0.0072055360578797256,
 'GH': 0.013076014124390205,
 'LULU': 0.005552330405890385,
 'MELI': 0.011117735373176117,
 'MNST': 0.0225681456974116,
 'MTCH': 0.012048934690333558,
 'NVCR': 0.004185312375554736,
 'QDEL': 0.01186589083079871,
 'ROKU': 0.028820691502474977,
 'TWLO': 0.008425828454778713,
 'VEEV': 0.010282416172917882}

In [26]:
retain_4 = {k: v * w[2] for k, v in retain_4.items()}
retain_4

{'ETSY': 0.02681301996376736,
 'ISRG': 0.07208296243063188,
 'JD': 0.029968347447177593,
 'RVLV': 0.0010430594535398474,
 'XPEV': 0.01536551509761973}